In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Subset

In [ ]:
from hartufo import CipicPlane
from hartufo import AnthropometrySpec, SubjectSpec
from hartufo.torch import collate_dict_dataset

In [ ]:
from sklearn.model_selection import GroupShuffleSplit

In [ ]:
import numpy as np

In [ ]:
plane = 'horizontal'
domain = 'magnitude'
side = 'any-left'
anthro_selection = ('head-torso', 'pinna-size', 'pinna-angle')

In [ ]:
ds = CipicPlane('../hartufo-collections/CIPIC', plane, domain, side, hrir_role='target',
    other_specs=dict(
        features_spec=AnthropometrySpec(select=anthro_selection),
        group_spec=SubjectSpec(),
    )
)

In [ ]:
num_inputs = len(ds.query.anthropometry_names(anthro_selection))
if domain == 'time':
    num_outputs = (len(ds.plane_angles), ds.hrir_length)
else:
    num_outputs = (len(ds.plane_angles), len(ds.hrtf_frequencies))
num_inputs, num_outputs

In [ ]:
splitter = GroupShuffleSplit(n_splits=1, random_state=0)
train_indices, test_indices = next(splitter.split(X=None, groups=ds.group))
train_dataset = Subset(ds, train_indices)
test_dataset = Subset(ds, test_indices)

In [ ]:
batch_size = len(train_dataset) // 4

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True, collate_fn=collate_dict_dataset)
test_dataloader = DataLoader(test_dataset, batch_size, collate_fn=collate_dict_dataset)

In [ ]:
class MLP(nn.Module):
    def __init__(self, num_inputs, num_outputs, num_hidden_layers, layer_size, activation=nn.ReLU):
        super().__init__()
        self.num_outputs = num_outputs
        def block(num_in, num_out, activation):
            return (
                nn.Linear(num_in, num_out),
                activation(),
            )
        self.model = nn.Sequential(
            *block(num_inputs, layer_size, activation),
            *[layer for _ in range(num_hidden_layers) for layer in block(layer_size, layer_size, activation)],
            nn.Linear(layer_size, np.prod(num_outputs)),
        )

    def forward(self, x):
        output = self.model(x)
        return output.view(-1, *self.num_outputs)

In [ ]:
device = 'cpu'
model = MLP(num_inputs, num_outputs, 5, 64).to(device)
print(model)

In [ ]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    current = 0
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        current += len(X)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 1 == 0:
            loss = loss.item()
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    num_batches = len(dataloader)
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
    test_loss /= num_batches
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")